In [ ]:
import os
import pandas as pd
import json
import pprint
import re
import nbformat

In [ ]:
# Define the path
path = os.path.join("GitHub", "DataProcessing")

# Print the path
print(path)


In [ ]:
pwd

In [ ]:
td = pd.read_json("..\DataProcessing\TestData\message.json", orient='index')

In [ ]:
# Read the JSON file
with open("../DataProcessing/TestData/message.json", "r") as file:
    data = json.load(file)

In [ ]:
# Access the 'participants' array within 'metadata'
participants_data = data['metadata']['participants']
participants_data

In [ ]:
# Get every timestamp within a game paired with corresponding minute
for counter, frame in enumerate(data['info']['frames']):
    timestamp = frame['timestamp']
    print(f"{counter + 1} Timestamp = {timestamp}")

In [ ]:
# Pull specific data for specified participant id at specific minute
def get_participant_data_at_minute(data, participant_id, target_minute, display_nozero=False):
    timestamp_data = []

    def display_nonzero_params(participant_frame):
        # Iterate through the participant frame's items
        for key, value in participant_frame.items():
            if isinstance(value, dict):
                print(f"{key}:")
                # If the value is a nested dictionary, iterate through its items
                display_nonzero_params(value)
            elif isinstance(value, (int, float)) and (not display_nozero or value != 0):
                # Display the parameter if it's a numeric value and not zero
                print(f"{key}: {value}")

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            current_minute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if current_minute == target_minute:
                participant_frames = frame.get('participantFrames', {})
                if participant_id in participant_frames:
                    participant_data = participant_frames[participant_id]
                    timestamp_data.append([timestamp, participant_data])

                    # Display nonzero parameters if specified
                    if display_nozero:
                        print(f"Timestamp at minute {target_minute}: {timestamp}")
                        display_nonzero_params(participant_data)
                        print("\n---\n")

    return timestamp_data

desired_participant_id = "6"
desired_minute = 10 

# Call the function with display_nozero set to True
result = get_participant_data_at_minute(data, desired_participant_id, desired_minute, display_nozero=True)


In [ ]:
# [NOTFINISHED] Get killed within specified timestamps 
def getParticipantIdByKill(data, start_timestamp, end_timestamp):
    for frame in data['info']['frames']:
        timestamp = frame['timestamp']
        
        if start_timestamp <= timestamp <= end_timestamp:
            for event in frame.get('events', []):
                victim_damage_dealt = event.get('victimDamageDealt', [])
                
                if victim_damage_dealt:
                    print(f"Timestamp: {timestamp}")
                    print("Victim Damage Dealt:")
                    for damage_info in victim_damage_dealt:
                        print(f"  Name: {damage_info['name']}")
            

# Example usage: Specify the start and end timestamps
start_timestamp = 120000  # Replace with your desired start timestamp
end_timestamp = 240000    # Replace with your desired end timestamp

# Call the function with the specified time frame
getParticipantIdByKill(data, start_timestamp, end_timestamp)


In [ ]:
# Get timestamp of every kill in the game
def get_timestamp_of_champion_kill(data):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                timestamp = event['timestamp']
                print(f"Timestamp of CHAMPION_KILL: {timestamp}")
        
                
# Call the function
get_timestamp_of_champion_kill(data)


In [ ]:
# [NOTFINSHED] Print out all champion kill information at specific kill timestamp
def get_event_of_champion_kill_at_timestamp(data, target_timestamp):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL' and event['timestamp'] == target_timestamp:
                return event

# Example: Get CHAMPION_KILL event data at timestamp 134226
target_timestamp = 134226
champion_kill_event = get_event_of_champion_kill_at_timestamp(data, target_timestamp)

# Print or use the champion_kill_event data as needed
if champion_kill_event:
    print(f"CHAMPION_KILL event data:")
    pprint.pprint(champion_kill_event)
else:
    print(f"No CHAMPION_KILL event found at timestamp {target_timestamp}.")
